<a href="https://colab.research.google.com/github/Danish-Hussain-DSAI/Deep-Fake-on-RESNET50/blob/main/capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

with zipfile.ZipFile('/content/drive/MyDrive/ds fake and real images/real.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/drive/MyDrive/extracted_real')

In [ ]:
import numpy as np
import tensorflow as tf


In [ ]:

folder_path="/content/drive/MyDrive/real"
dataset=tf.keras.utils.image_dataset_from_directory(
    folder_path,
    image_size=(64,64),
    batch_size=32,







                                                    )
dataset_shape = dataset.element_spec[0].shape
print("Shape of the dataset:", dataset_shape)

Found 15387 files belonging to 2 classes.
Shape of the dataset: (None, 64, 64, 3)


In [ ]:
classes=dataset.class_names
print(classes)


['fake', 'real']


In [ ]:
from tensorflow.keras.layers import Lambda

normalize = Lambda(lambda x: x / 255.0)



dataset = dataset.map(
    lambda x, y: (normalize(x), y)
)



In [ ]:
# Define the size of the training set (e.g., 80% for training)
train_size = 0.8

# Calculate the number of batches for training and testing
num_batches = len(dataset)
num_train_batches = int(train_size * num_batches)
num_test_batches = num_batches - num_train_batches

# Split the dataset into training and testing sets
train_dataset = dataset.take(num_train_batches)
test_dataset = dataset.skip(num_train_batches)

# Print the sizes of the training and testing sets
print(f"Training set size: {num_train_batches} batches")
print(f"Testing set size: {num_test_batches} batches")


Training set size: 384 batches
Testing set size: 97 batches


In [ ]:
# one-hot encoding
train_dataset = train_dataset.map(lambda x, y: (x, tf.one_hot(y, depth=2)))
test_dataset = test_dataset.map(lambda x, y: (x, tf.one_hot(y, depth=2)))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

# Define your data augmentation transformations
data_augmentation = tf.keras.Sequential([
    preprocessing.Rescaling(1./255),  # Rescale pixel values to [0, 1]
    preprocessing.RandomFlip("horizontal"),  # Randomly flip images horizontally
    preprocessing.RandomRotation(0.2),  # Randomly rotate images by up to 20 degrees
    preprocessing.RandomZoom(0.2),  # Randomly zoom images by up to 20%
    preprocessing.RandomContrast(0.2),  # Randomly adjust image contrast
])

# Apply data augmentation to your original dataset and create augmented dataset
augmented_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))

# Concatenate the augmented data with the original dataset
combined_dataset = train_dataset.concatenate(augmented_dataset)

# Shuffle the combined dataset
train_dataset = combined_dataset.shuffle(buffer_size=1000)  # You can adjust the buffer_size as needed

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

# Define the ResNet model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Add custom layers on top of the ResNet base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)

x = Dropout(0.5)(x)  # Adding dropout
x = BatchNormalization()(x)  # Adding batch normalization
predictions = Dense(2, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Optionally, freeze the layers of the ResNet base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Create early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Train the model on your training dataset and collect history
history = model.fit(train_dataset, epochs=5, batch_size=32, validation_data=test_dataset, callbacks=[early_stopping])

# Evaluate the model on your testing dataset
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/5
768/768 [==============================] - 83s 49ms/step - loss: 0.6782 - accuracy: 0.6201 - val_loss: 0.6245 - val_accuracy: 0.6986
Epoch 2/5
768/768 [==============================] - 97s 73ms/step - loss: 0.6218 - accuracy: 0.7074 - val_loss: 0.6096 - val_accuracy: 0.6993
Epoch 3/5
768/768 [==============================] - 99s 73ms/step - loss: 0.6069 - accuracy: 0.7111 - val_loss: 0.6112 - val_accuracy: 0.6986
Epoch 4/5
768/768 [==============================] - 74s 45ms/step - loss: 0.6061 - accuracy: 0.7111 - val_loss: 0.6090 - val_accuracy: 0.7022
Epoch 5/5
97/97 [==============================] - 22s 47ms/step - loss: 0.6120 - accuracy: 0.6993
Test Accuracy: 69.93%


In [ ]:
model.save('model.h5') # Saves the entire model to a single artifact

In [ ]:
from matplotlib.rcsetup import validate_color
# Plot training and validation accuracy
plt.figure(figsize=(10, 6))
plt.plot(accuracy, label='Training Accuracy')
plt.plot(validate_color_accuracy, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

NameError: ignored

<Figure size 1000x600 with 0 Axes>